# The Sims-Flanagan gradients
This notebook is the result of a few days of blood shed with manual derivatives assembly. 
It benchmarks a python version of the computation of a sims-flanagan leg and the same done using heyoka.

In [1]:
import heyoka as hy
import numpy as np
import pykep as pk

In [2]:
# Problem data
mu = pk.MU_SUN
max_thrust = 0.12
isp = 100
veff = isp * 9.80665

# Leg dimension
nseg=5
nseg_fwd=3
nseg_bck=2

# Initial state
ms = 1500.0
rs = np.array([1, 0.1, -0.1]) * pk.AU
vs = np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY

# Final state
mf = 1300.0
rf = np.array([1.2, -0.1, 0.1]) * pk.AU
vf = np.array([-0.2, 1.023, -0.44]) * pk.EARTH_VELOCITY

# Thrust
throttles = np.array([0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2,  0.21, 0.22, 0.23, 0.24])
tof = 324.0 * pk.DAY2SEC

We first build the leg in the symbolic heyoka manipulator.

In [3]:
def propagate_lagrangian_heyoka(pos_0, vel_0, mu, tof):
    x0, y0, z0 = pos_0
    vx0, vy0, vz0 = vel_0
    v02 = vx0**2 + vy0**2 + vz0**2
    r0 = hy.sqrt(x0**2 + y0**2 + z0**2)
    eps = v02 * 0.5 - mu / r0
    a = -mu / (2.0 * eps)

    sigma0 = np.dot(pos_0, vel_0) / np.sqrt(mu)
    s0 = sigma0 / hy.sqrt(a)
    c0 = 1.0 - r0 / a

    n = hy.sqrt(mu / (a * a * a))
    DM = n * tof

    DE = hy.kepDE(s0, c0, DM)

    # Compute cos(DE) and sin(DE).
    cDE = hy.cos(DE)
    sDE = hy.sin(DE)

    r = a + (r0 - a) * cDE + sigma0 * hy.sqrt(a) * sDE

    F = 1.0 - a / r0 * (1.0 - cDE)
    G = a * sigma0 / np.sqrt(mu) * (1.0 - cDE) + r0 * hy.sqrt(a / mu) * sDE
    Ft = -hy.sqrt(mu * a) / (r * r0) * sDE
    Gt = 1 - a / r * (1.0 - cDE)

    pos = F * pos_0 + G * vel_0
    vel = Ft * pos_0 + Gt * vel_0

    return [pos, vel]

# Forward computation for the mismatch state
def multiple_impulses_heyoka(xs, ys, zs, vxs, vys, vzs, ms, throttles_vars, tof, veff):
    nseg = int(len(throttles_vars) / 3)
    dt = tof / nseg
    c = max_thrust * dt
    
    pos = np.array([xs, ys, zs])
    vel = np.array([vxs, vys, vzs])
    m=ms

    for i in range(nseg):
        ux, uy, uz = throttles_vars[3*i:3*i+3]
        if i == 0:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
        else:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt)  
        vel = vel + np.array([ux, uy, uz]) * c / m
        m = m * hy.exp(-hy.sqrt(ux**2+uy**2+uz**2) * c / veff / m)
    # 4
    pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
    retval = np.hstack([pos, vel, m])
    return retval

Lets define the symbolic variables

In [4]:
# The symbolic variables
xs_var, ys_var, zs_var = hy.make_vars("xs", "ys", "zs")
vxs_var, vys_var, vzs_var = hy.make_vars("vxs", "vys", "vzs")
ms_var, veff_var= hy.make_vars("ms", "veff")

xf_var, yf_var, zf_var = hy.make_vars("xf", "yf", "zf")
vxf_var, vyf_var, vzf_var = hy.make_vars("vxf", "vyf", "vzf")
mf_var = hy.make_vars("mf")

tof_var = hy.make_vars("t")
throttles_symbols = []
for i in range(nseg):
    throttles_symbols.extend(["ux"+str(i), "uy"+str(i), "uz"+str(i)])

throttles_vars = hy.make_vars(*throttles_symbols)
throttles_vars_fwd=throttles_vars[:nseg_fwd*3]
throttles_vars_bck=throttles_vars[nseg_fwd*3:]
throttles_vars_bck.reverse()
for i in range(nseg_bck):
    throttles_vars_bck[3*i:3*i+3] = throttles_vars_bck[3*i:3*i+3][::-1]

In [5]:
# Here we define the symbolic function
xf_fwd = multiple_impulses_heyoka(xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, throttles_vars_fwd, tof_var/nseg*nseg_fwd, veff_var)
xf_bck = multiple_impulses_heyoka(xf_var, yf_var, zf_var, -vxf_var, -vyf_var, -vzf_var, mf_var, throttles_vars_bck, tof_var/nseg*nseg_bck, -veff_var)
xf_bck[3:6]*=-1
mc = xf_fwd - xf_bck

... and compile them into compiled functions

In [6]:
mc_cf = hy.cfunc(mc, vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, xf_var, yf_var, zf_var, vxf_var, vyf_var, vzf_var, mf_var, *throttles_vars, tof_var, veff_var])

In [7]:
print("mc:", mc_cf(list(rs)+list(vs)+[ms]+list(rf)+list(vf)+[mf]+ throttles.tolist() + [tof, veff]))

mc: [-1.97012748e+11  4.69650442e+11 -1.50075233e+11 -2.99751515e+04
 -2.82649162e+04  1.02648068e+04 -8.28076734e+02]


... and the same for the gradients

In [8]:
dtens1 = hy.diff_tensors(mc,
                    diff_args=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var],
                    diff_order=1
                    )

dtens2 = hy.diff_tensors(mc,
                    diff_args=[xf_var, yf_var, zf_var, vxf_var, vyf_var, vzf_var, mf_var],
                    diff_order=1
                    )

dtens3 = hy.diff_tensors(mc,
                    diff_args=[*throttles_vars, tof_var],
                    diff_order=1
                    )

jac1 = dtens1.jacobian   
jac2 = dtens2.jacobian   
jac3 = dtens3.jacobian   


mcg_xs_hy_func = hy.cfunc(jac1.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, xf_var, yf_var, zf_var, vxf_var, vyf_var, vzf_var, mf_var, *throttles_vars, tof_var, veff_var])

mcg_xf_hy_func = hy.cfunc(jac2.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, xf_var, yf_var, zf_var, vxf_var, vyf_var, vzf_var, mf_var, *throttles_vars, tof_var, veff_var])

mcg_th_tof_hy_func = hy.cfunc(jac3.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, xf_var, yf_var, zf_var, vxf_var, vyf_var, vzf_var, mf_var, *throttles_vars, tof_var, veff_var])

In [10]:
%%timeit
mcg_xs_hy_func(list(rs)+list(vs)+[ms]+list(rf)+list(vf)+[mf]+ throttles.tolist() + [tof, veff])

5.25 μs ± 42.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
mcg_xs_hy = mcg_xs_hy_func(list(rs)+list(vs)+[ms]+list(rf)+list(vf)+[mf]+ throttles.tolist() + [tof, veff]).reshape(7, -1)
mcg_xf_hy = mcg_xf_hy_func(list(rs)+list(vs)+[ms]+list(rf)+list(vf)+[mf]+ throttles.tolist() + [tof, veff]).reshape(7, -1)
mcg_th_tof_hy = mcg_th_tof_hy_func(list(rs)+list(vs)+[ms]+list(rf)+list(vf)+[mf]+ throttles.tolist() + [tof, veff]).reshape(7, -1)

we set a sf leg from the c++ code

In [12]:
sf_leg = pk.leg.sims_flanagan()
sf_leg.rvs = [rs, vs]
sf_leg.rvf = [rf, vf]
sf_leg.tof = tof
sf_leg.ms=ms
sf_leg.mf=mf
sf_leg.throttles=throttles
sf_leg.mu = mu
sf_leg.max_thrust=max_thrust
sf_leg.isp=isp
sf_leg.cut=0.6

In [13]:
err = (mc_cf([*rs,*vs,ms,*rf,*vf,mf,*throttles,tof,veff])-np.array(sf_leg.compute_mismatch_constraints()))/np.array(sf_leg.compute_mismatch_constraints())
print("Different in mismatch constraints between heyoka cfunc and the c++ code", err)

Different in mismatch constraints between heyoka cfunc and the c++ code [ 3.09803080e-16 -2.59917380e-16 -8.13394122e-16 -2.42732972e-16
 -5.14840205e-16  0.00000000e+00 -0.00000000e+00]


In [15]:
mcg_xs, mcg_xf, mcg_th_tof = sf_leg.compute_mc_grad()
eps = 1e-30
print("Max difference on mcg_xs: ", np.max(np.abs((mcg_xs-mcg_xs_hy) / (mcg_xs_hy+eps))))
print("Max difference on mcg_xf: ", np.max(np.abs((mcg_xf-mcg_xf_hy) / (mcg_xf_hy+eps))))
print("Max difference on mcg_th_tof: ", np.max(np.abs((mcg_th_tof-mcg_th_tof_hy) / (mcg_th_tof_hy+eps))))

Max difference on mcg_xs:  2.5079055870093578e-14
Max difference on mcg_xf:  6.333256510692337e-15
Max difference on mcg_th_tof:  9.742677536541831e-14


Coe to estimate numerically

In [16]:
import pygmo as pg
def mc(x):
    sf_leg.rvf = [x[:3], x[3:6]]
    sf_leg.mf = x[6]
    return sf_leg.compute_mismatch_constraints()
pg.estimate_gradient_h(mc, [*rf,*vf,mf])

array([-2.18195555e+00,  7.31859658e-01, -4.01041457e-01,  1.29887813e+07,
       -2.57342254e+06,  1.27466361e+06,  8.39381051e+05,  1.00244597e+00,
       -7.64056237e-01,  2.58933482e-01, -2.93141622e+06,  1.15461634e+07,
       -1.17307810e+06,  8.16365590e+05, -5.15896647e-01,  2.46191906e-01,
       -3.94008353e-01,  1.42698746e+06, -1.15655018e+06,  9.66632480e+06,
        8.97445464e+05,  1.32836662e-07, -1.02924961e-07,  5.43186030e-08,
       -1.18665018e+00,  5.08767181e-01, -2.44188177e-01, -1.22273512e-01,
       -1.80621026e-07,  7.65340442e-09, -5.10854337e-08,  6.35027810e-01,
       -1.25653396e+00,  3.13309785e-01, -1.18516807e-01,  8.73770616e-08,
       -4.74593633e-08, -6.83509638e-08, -2.98022301e-01,  3.07553676e-01,
       -7.34518838e-01, -1.30918860e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -9.62784376e-01])

# Non Heyoka
We perform the same computations not using heyoka, but numerically

In [17]:
# The dynamics
def dyn(r, v, mu):
    R3 = np.linalg.norm(r) ** 3
    f = [v[0], v[1], v[2], -mu / R3 * r[0], -mu / R3 * r[1], -mu / R3 * r[2]]
    return np.array(f).reshape(-1, 1)


# This is the main computation ported to C++. Manual assembly of the gradients.
def gradients_multiple_impulses(xs, ys, zs, vxs, vys, vzs, ms, throttles, tof, veff):
    nseg = int(throttles.shape[0] / 3)
    c = max_thrust * tof / nseg
    a = 1.0 / veff
    dt = tof / nseg
    m0 = ms

    # We define the indepedent variables
    u = []
    un = []
    du = []
    m = [m0]

    for i in range(nseg):
        u.append(np.array(throttles[i * 3 : i * 3 + 3]).reshape(1, 3))
        du.append(np.zeros((3, 3 * nseg + 2)))
        du[i][:, 3 * i : 3 * i + 3] = np.eye(3)

    dm = [np.hstack((np.zeros((1, 3 * nseg)), np.eye(1), np.zeros((1, 1))))]
    dtof = np.hstack((np.zeros((1, 3 * nseg + 1)), np.eye(1)))
    Dv = []
    dDv = []

    # 1 - We compute the mass schedule and related quantities

    for i in range(nseg):
        Dv.append(c / m[i] * u[i])
        un = np.sqrt(u[i][0][0] ** 2 + u[i][0][1] ** 2 + u[i][0][2] ** 2)
        Dvn = c / m[i] * un
        dDv.append(
            c / m[i] * du[i]
            - c / m[i] ** 2 * u[i].T @ dm[i]
            + max_thrust / m[i] * u[i].T @ dtof / nseg
        )
        dDvn = (
            c / m[i] / un * u[i] @ du[i]
            - c / m[i] ** 2 * un * dm[i]
            + max_thrust / m[i] * un * dtof / nseg
        )
        m.append(m[i] * np.exp(-Dvn * a))
        dm.append(-m[i + 1] * a * dDvn + np.exp(-Dvn * a) * dm[i])

    M = []
    f = []
    rv_it = [[xs, ys, zs], [vxs, vys, vzs]]
    tofs = [dt] * (nseg + 1)
    tofs[0] /= 2.0
    tofs[-1] /= 2.0

    # 2 - We compute the state transition matrices
    for i in range(nseg + 1):
        rv_it, M_it = pk.propagate_lagrangian(rv=rv_it, tof=tofs[i], mu=mu, stm=True)
        M.append(M_it)
        # We compute f (before impulse)
        r = rv_it[0]
        f.append(dyn(r, rv_it[1], mu))
        # And add the impulse if needed
        if i < nseg:
            rv_it[1] = [a + b for a, b in zip(rv_it[1], Dv[i].flatten())]

    # 3 - We assemble the gradients
    Iv = np.diag((0, 0, 0, 1, 1, 1))[:, 3:]

    # Mc will contain [Mn@..@M0,Mn@..@M1, Mn]
    Mc = [0] * (nseg + 1)
    Mc[-1] = M[-1]
    for i in range(1, len(M)):
        Mc[-1 - i] = Mc[-1 - i + 1] @ M[-1 - i]

    # grad_tof
    grad_tof = 0.5 * f[-1]
    for i in range(nseg - 1):
        grad_tof += Mc[i + 2] @ f[i + 1]

    grad_tof += 0.5 * Mc[1] @ f[0]
    grad_tof /= nseg
    for i in range(nseg):
        grad_tof += Mc[i + 1] @ Iv @ dDv[i][:, -1:]

    # grad u
    grad_u = 0
    for i in range(nseg):
        grad_u += Mc[i + 1] @ Iv @ dDv[i][:, :-2]

    # grad ms
    grad_ms = 0
    for i in range(nseg):
        grad_ms += Mc[i + 1] @ Iv @ dDv[i][:, -2:-1]

    # grad xs
    grad_xs = Mc[0]

    # Assembling te return value
    # grad will contain the gradient of the final posvelm with respect to the throttles and the tof
    grad = np.hstack((grad_u, grad_tof))
    grad = np.vstack((grad, dm[-1][:, :-1]))
    grad[-1, -1] = dm[-1][:, -1][0]
    # grad_ic will contain the gradient of the final posvelm with respect to the initial posvelm
    grad_ic = np.vstack((grad_xs, np.zeros((1, 6))))
    grad_ic = np.hstack((grad_ic, np.zeros((7, 1))))
    grad_ic[:6, -1:] = grad_ms
    grad_ic[-1, -1] = dm[-1][:, -2][0]
    return (grad, grad_ic)


def gradients_fwd(xs, ys, zs, vxs, vys, vzs, ms, throttles_fwd, tof, veff):
    return gradients_multiple_impulses(xs, ys, zs, vxs, vys, vzs, ms, throttles_fwd, tof, veff)


def gradients_bck(xf, yf, zf, vxf, vyf, vzf, mf, throttles_bck, tof, veff):
    throttles_bck_l = list(throttles_bck)
    throttles_bck_l.reverse()
    for i in range(int(len(throttles_bck_l) / 3)):
        throttles_bck_l[3 * i : 3 * i + 3] = throttles_bck_l[3 * i : 3 * i + 3][::-1]

    grad, grad_ic = gradients_multiple_impulses(xf, yf, zf, -vxf, -vyf, -vzf, mf, np.array(throttles_bck_l), tof, -veff)
    grad_ic[3:6, :] *= -1
    grad_ic[:, 3:6] *= -1
    grad[3:6, :] *= -1
    grad[:, :-1]*=-1

    # Note that the throttles in xgrad are ordered in reverse. Before returning we must restore the forward order
    grad[:, :-1] = np.flip(grad[:, :-1], axis=1)
    for i in range(int(len(throttles_bck_l) / 3)):
        grad[:, 3 * i : i * 3 + 3] = np.flip(grad[:, 3 * i : i * 3 + 3], axis=1)

    # We return
    return (grad, grad_ic)


def grad_mc(rs, vs, ms, rf, vf, mf, throttles, tof):
    grad_fwd, grad_ic_fwd = gradients_fwd(*rs, *vs, ms, np.array(throttles[: nseg_fwd * 3]), tof / nseg * nseg_fwd, veff)
    grad_bck, grad_ic_bck = gradients_bck(*rf, *vf, mf, np.array(throttles[nseg_fwd * 3 :]), tof / nseg * nseg_bck, veff)
    return (grad_fwd, grad_bck, grad_ic_bck)

In [18]:
%%timeit
grad_mc(list(rs),list(vs),ms,list(rf),list(vf),mf, throttles, tof)

220 μs ± 1.81 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Let us perform the computation and check that is the same

In [19]:
fwd, bck, ic_bck = grad_mc(list(rs),list(vs),ms,list(rf),list(vf),mf, throttles, tof)
# This is the actual gradient to compare with the c++ code
res = np.hstack((fwd[:,:-1], bck[:,:-1], fwd[:,-1:]*nseg_fwd/nseg - bck[:,-1:]*nseg_bck/nseg, ic_bck[:,-1:]))
# This is without the last column mf
res_no_hy = np.hstack((fwd[:,:-1], bck[:,:-1], fwd[:,-1:]*nseg_fwd/nseg - bck[:,-1:]*nseg_bck/nseg,))

In [20]:
# In the C++ we have non dimensional units for the mc so we need to rescale the gradient too
res[:3,:]/=pk.AU
res[3:6,:]/=pk.EARTH_VELOCITY
res[7:,:]/=1e8
res[:, -2] = res[:, -2] * pk.DAY2SEC
print("Compare this to the result of the C++ code in the leg_sims_flanagan_test.cpp (grad_test): ", res[0,:])

Compare this to the result of the C++ code in the leg_sims_flanagan_test.cpp (grad_test):  [ 3.27858926e-02  9.23774757e-03 -6.27370268e-04  2.42975689e-02
 -5.16519045e-04  6.38519017e-04  1.02031523e-02 -1.58197319e-04
  2.34809677e-05 -7.81363717e-03  4.77353349e-05 -2.14219032e-05
 -2.90780417e-02  3.56703937e-03 -9.91240785e-04 -2.92171589e-03
 -5.61091576e-06]


Here we compare the heyoka to the numerical computations.

In [21]:
(res_no_hy-mcg_th_tof_hy)/mcg_th_tof_hy

array([[ 3.88881836e-16, -8.62618161e-16, -6.35083397e-15,
         1.57421158e-15,  2.19843289e-14,  3.43197049e-15,
         2.34299614e-15, -1.76300528e-14, -7.05245564e-14,
        -2.03967721e-16,  8.35974007e-14,  6.56787841e-14,
        -0.00000000e+00,  2.79245895e-15,  6.23027515e-15,
        -3.59567572e-16],
       [ 1.32469822e-15,  9.58542907e-16,  1.38949418e-15,
        -7.77650104e-14,  8.83208958e-16,  3.14926601e-15,
        -4.74314201e-14,  1.05132106e-15,  1.61431258e-14,
        -6.12423716e-15, -0.00000000e+00,  5.38695304e-15,
         4.32155119e-16, -4.25460174e-16, -1.74935833e-16,
         1.83421533e-14],
       [ 2.41725018e-15,  1.62513618e-15, -5.55242706e-16,
        -6.46724494e-15,  2.29024779e-15,  1.30505730e-16,
        -9.74267754e-14,  1.04130477e-14,  7.84480153e-16,
         5.06696393e-14, -4.42022831e-14, -2.04039306e-16,
         7.69899046e-16, -4.55402714e-15, -4.73758448e-16,
        -4.29941981e-15],
       [-2.20526621e-14,  7.47781343e